## Notebook to calculate all important metrics for the networks and save them

In [36]:
import pandas as pd
import geopandas as gpd
import numpy as np
import pypsa

# imported own functions
from utils import market_values, capacity, capacity_links, capacity_storage_units, generation, generation_links, \
    generation_storage_units, market_values_storage_units, market_values_links, time_stored_LIFO

# imported own definitions
from utils import resistive_heater, gas_boiler, heat_pump, water_tanks_charger, water_tanks_discharger, solar_thermal

# general variables
onshore_regions = gpd.read_file("../data/external/regions_onshore_elec_s_181.geojson")
offshore_regions = gpd.read_file("../data/external/regions_offshore_elec_s_181.geojson")
onshore_regions = onshore_regions.set_index('name')
offshore_regions = offshore_regions.set_index('name')

# Regions
onshore_regions['coords'] = onshore_regions['geometry'].apply(lambda x: x.representative_point().coords[:])
onshore_regions['coords'] = [coords[0] for coords in onshore_regions['coords']]
onshore_regions["name"] = onshore_regions.index
offshore_regions['coords'] = offshore_regions['geometry'].apply(lambda x: x.representative_point().coords[:])
offshore_regions['coords'] = [coords[0] for coords in offshore_regions['coords']]
offshore_regions["name"] = offshore_regions.index

# carrier lists
from utils import c_el_gen_s, c_el_con_s

# Network imports
stst = pypsa.Network("../data/raw/elec_s_181_lv1.0__Co2L0-3H-T-H-B-I-A-solar+p3-linemaxext10-noH2network_2030.nc")
exp = pypsa.Network("../data/raw/elec_s_181_lvopt__Co2L0-3H-T-H-B-I-A-solar+p3-linemaxext10_2030.nc")

# Notebook Definitions
c1_groups = [resistive_heater, gas_boiler, heat_pump, water_tanks_charger, water_tanks_discharger, solar_thermal]
c1_groups_name = ["resistive heater", "gas boiler", "heat pump", "water tanks charger", "water tanks discharger",
                  "solar thermal"]

Importing PyPSA from older version of PyPSA than current version.
Please read the release notes at https://pypsa.readthedocs.io/en/latest/release_notes.html
carefully to prepare your network for import.
Currently used PyPSA version [0, 20, 1], imported network file PyPSA version [0, 18, 1].

INFO:pypsa.io:Imported network elec_s_181_lv1.0__Co2L0-3H-T-H-B-I-A-solar+p3-linemaxext10-noH2network_2030.nc has buses, carriers, generators, global_constraints, lines, links, loads, storage_units, stores
Importing PyPSA from older version of PyPSA than current version.
Please read the release notes at https://pypsa.readthedocs.io/en/latest/release_notes.html
carefully to prepare your network for import.
Currently used PyPSA version [0, 20, 1], imported network file PyPSA version [0, 18, 1].

INFO:pypsa.io:Imported network elec_s_181_lvopt__Co2L0-3H-T-H-B-I-A-solar+p3-linemaxext10_2030.nc has buses, carriers, generators, global_constraints, lines, links, loads, storage_units, stores


## Spatial dfs

In [37]:
# calc market values, generation, lmps, capacity factors for generators, links and storage units
for n in [stst, exp]:
    df_regions_onshore = onshore_regions.copy()
    df_regions_offshore = offshore_regions.copy()

    # function for carriers in n.generators.carrier.unique() # 13 / 13
    for carrier in n.generators.carrier.unique():
        df_regions_onshore[f"{carrier}_mv"] = market_values(n, carrier)
        df_regions_offshore[f"{carrier}_mv"] = market_values(n, carrier)
        df_regions_onshore[f"{carrier}_gen"] = generation(n, carrier) / 1000 * 3
        df_regions_offshore[f"{carrier}_gen"] = generation(n, carrier) / 1000 * 3
        # lmps
        # capacity factors This calculation is correct? as capacity is multiplied by 2920 is the same as multiplying the generation by 3 and then dividing it by the capacity times 8760 (as cap is in MWh?)
        df_regions_onshore[f"{carrier}_cf"] = generation(n, carrier) / (capacity(n, carrier) * 2920)
        df_regions_offshore[f"{carrier}_cf"] = generation(n, carrier) / (capacity(n, carrier) * 2920)

    # function for carriers in n.links.carrier.unique() # 53 / 55
    for carrier in n.links.carrier.unique():
        df_regions_onshore[f"{carrier}_mv"] = market_values_links(n, carrier)
        df_regions_onshore[f"{carrier}_gen"] = generation_links(n, carrier) / 1000 * 3
        df_regions_onshore[f"{carrier}_cf"] = generation_links(n, carrier) / (capacity_links(n, carrier) * 2920)

    # function for carriers in n.storage_units.carrier.unique() # 2 / 2
    for carrier in n.storage_units.carrier.unique():
        df_regions_onshore[f"{carrier}_mv"] = market_values_storage_units(n, carrier)
        df_regions_onshore[f"{carrier}_gen"] = generation_storage_units(n, carrier) / 1000 * 3
        # capacity factors (both generation and consumption(loading) is considered
        gen = abs(n.storage_units_t.p.loc[:, n.storage_units.carrier == carrier])
        gen.columns = gen.columns.map(n.storage_units.bus)
        gen.columns = gen.columns.map(n.buses.location)
        df_regions_onshore[f"{carrier}_cf"] = gen.sum() / (capacity_storage_units(n, carrier) * 2920)

    # set market values to nan where generation in corresponding region is lower than % quantile
    qt = 0.2
    for carrier in (n.generators.carrier.unique().tolist() +
                    n.links.carrier.unique().tolist() +
                    n.storage_units.carrier.unique().tolist()):
        index = df_regions_onshore[f"{carrier}_gen"] <= np.nanquantile(df_regions_onshore[f"{carrier}_gen"], qt)
        df_regions_onshore[f"{carrier}_mv"][index] = np.nan

    # calc lmps at the buses (lmps that are only present for EU (e.g. oil) are nan at the moment)
    # TODO: decide if EU lmps are used as lmp for all regions
    for carrier_bus in n.buses.carrier.unique():
        # index would be same names as the bus (not the location as it is in the index of
        # df_regions_onshore so far -> map location to make sure the right lmp is set
        locs = n.buses.location[n.buses[n.buses.carrier == carrier_bus].index]
        lmps = n.buses_t.marginal_price[n.buses[n.buses.carrier == carrier_bus].index].mean()
        df = pd.concat([lmps, locs], axis=1).rename(columns={0: f"{carrier_bus}_lmp"})
        df.set_index("location", inplace=True)
        if df.size == 1:
            if df.index == "EU":
                df = pd.DataFrame(np.repeat(df.values, 181), index=df_regions_onshore.index,
                                  columns=[f"{carrier_bus}_lmp"])
                df_regions_onshore[f"{carrier_bus}_lmp"] = df[f"{carrier_bus}_lmp"]
        else:
            df_regions_onshore[f"{carrier_bus}_lmp"] = df[f"{carrier_bus}_lmp"]

    if n == stst:
        df_stst_ons = df_regions_onshore
        df_stst_off = df_regions_offshore

    if n == exp:
        df_exp_ons = df_regions_onshore
        df_exp_off = df_regions_offshore

C:\Users\Julian\Programs\anaconda3\envs\ma_env\lib\site-packages\geopandas\geodataframe.py:1443: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

C:\Users\Julian\Programs\anaconda3\envs\ma_env\lib\site-packages\geopandas\geodataframe.py:1443: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

C:\Users\Julian\Programs\anaconda3\envs\ma_env\lib\site-packages\geopandas\geodataframe.py:1443: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Con

In [38]:
# consumed electric energy and prices payed
# AC and low voltage are the main buses for electricity (why the difference?) Investigate in differences in lmps?

for n , df in zip([stst, exp], [df_stst_ons, df_exp_ons]):

    for c in c_el_con_s:

        # links
        if c in n.links.carrier.unique().tolist():
            # check if bus 0 is AC or low voltage bus
            if n.links[n.links.carrier == c].bus0.map(n.buses.carrier).unique() in ["AC", "low voltage"]:

                # consumption of link at bus 0
                con = n.links_t.p0.loc[:, n.links.carrier == c]
                con.columns = con.columns.map(n.links.bus0)
                # save consumption per location to df
                con_sum = con.sum()
                con_sum.index = con_sum.index.map(n.buses.location)
                # convert to TWh and make negative
                df[f"{c}_con_el"] = con_sum / 1000 * 3 * -1
                # get lmp of buses where the links consumes from
                lmp_con = n.buses_t.marginal_price.loc[:, con.columns]
                # calculate costs for every time step and location
                overall_cost = con * lmp_con
                # calc consumption weighted average per location: overall cost per location / overall generation per location
                cost_mv = overall_cost.sum() / con.sum()
                cost_mv.index = cost_mv.index.map(n.buses.location)
                # save cost_mv to df (€/MWH_el)
                df[f"{c}_cost_mv_el"] = cost_mv

        # storage unit: only PHS can store
        elif c in n.storage_units.carrier.unique().tolist():

                # consumption of su
                con = n.storage_units_t.p_store.loc[:, n.storage_units.carrier == c]
                con.columns = con.columns.map(n.storage_units.bus)
                # save consumption per location to df in TWh and make negative
                df[f"{c}_con_el"] = con.sum() / 1000 * 3 * -1
                # get lmp of buses where the su consumes from
                lmp_con = n.buses_t.marginal_price.loc[:, con.columns]
                # calculate costs for every time step and location
                overall_cost = con * lmp_con
                # calc consumption weighted average per location: overall cost per location / overall generation per location
                cost_mv = overall_cost.sum() / con.sum()
                # save cost_mv to df
                df[f"{c}_cost_mv_el"] = cost_mv

        else:
            print(f"{c} not found!")

        # set pries to nan where generation in corresponding region is lower than % quantile
        qt = 0.2
        index = df[f"{c}_con_el"] <= np.nanquantile(df[f"{c}_con_el"], qt)
        df[f"{c}_cost_mv_el"][index] = np.nan

C:\Users\Julian\Programs\anaconda3\envs\ma_env\lib\site-packages\geopandas\geodataframe.py:1443: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

C:\Users\Julian\Programs\anaconda3\envs\ma_env\lib\site-packages\geopandas\geodataframe.py:1443: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

C:\Users\Julian\AppData\Local\Temp\ipykernel_17624\1488996566.py:54: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/use

In [39]:
df_stst_ons.head()

,geometry,coords,name,offwind-ac_mv,offwind-ac_gen,offwind-ac_cf,onwind_mv,onwind_gen,onwind_cf,solar_mv,...,BEV charger_con_el,BEV charger_cost_mv_el,battery charger_con_el,battery charger_cost_mv_el,urban central air heat pump_con_el,urban central air heat pump_cost_mv_el,urban central resistive heater_con_el,urban central resistive heater_cost_mv_el,PHS_con_el,PHS_cost_mv_el
name,,,,,,,,,,,,,,,,,,,,,
AL0 0,"POLYGON ((20.32207 39.91318, 20.39703 39.81809...","(20.00648437943388, 41.14563914300007)",AL0 0,NaN,0.029238,0.090705,NaN,0.015125,0.038018,44.628736,...,-2904.296308,62.673081,-10501.207286,NaN,-161.142629,142.000494,-6.587224,51.370506,NaN,NaN
AT0 0,"POLYGON ((16.45660 47.41184, 16.43396 47.39685...","(15.700718667666258, 47.896990619088925)",AT0 0,NaN,NaN,NaN,83.457109,30395.904252,0.149667,54.679083,...,-11920.650630,NaN,-9.661850,66.778969,-1892.970068,96.915190,-1366.630022,NaN,NaN,NaN
AT0 1,"POLYGON ((10.08288 47.35907, 10.20928 47.37248...","(10.379200993764837, 47.18231496200006)",AT0 1,NaN,NaN,NaN,NaN,0.000348,0.004685,65.745903,...,-1495.945182,105.445665,-1880.142837,75.537145,-3.673844,90.315217,-108.354978,54.638572,-2221.062727,NaN
AT0 2,"POLYGON ((13.47802 46.56357, 13.23111 46.55217...","(13.805469952867842, 47.57283376100011)",AT0 2,NaN,NaN,NaN,NaN,0.014320,0.039178,58.223787,...,-7323.647076,96.649590,-5879.983158,NaN,-668.225567,92.091020,-777.763461,33.551977,-4907.238190,NaN
BA0 0,"POLYGON ((17.82716 42.85312, 17.81176 42.90986...","(17.97055162265331, 43.88773569750005)",BA0 0,NaN,NaN,NaN,NaN,0.014059,0.031802,51.160537,...,-2996.008222,58.325767,-4823.635061,NaN,-177.316568,96.225890,-96.014816,38.225192,NaN,NaN


In [40]:
df_stst_off.head()

,geometry,coords,name,offwind-ac_mv,offwind-ac_gen,offwind-ac_cf,onwind_mv,onwind_gen,onwind_cf,solar_mv,...,services urban decentral solar thermal_cf,urban central solar thermal_mv,urban central solar thermal_gen,urban central solar thermal_cf,oil_mv,oil_gen,oil_cf,solar rooftop_mv,solar rooftop_gen,solar rooftop_cf
name,,,,,,,,,,,,,,,,,,,,,
AL0 0,"POLYGON ((19.98542 39.69475, 20.01003 39.69120...","(19.04227857401803, 40.790848115835146)",AL0 0,126.148417,0.029238,0.090705,131.075254,0.015125,0.038018,44.628736,...,0.049208,45.096866,73.985333,0.038386,NaN,NaN,NaN,56.499070,2987.307119,0.129410
BE0 0,"POLYGON ((3.25367 51.34858, 3.10403 51.31485, ...","(2.728512798889029, 51.5008335)",BE0 0,78.122119,6139.041623,0.393274,73.103671,9140.702915,0.312504,58.613241,...,0.025145,34.889495,0.044487,0.027616,NaN,NaN,NaN,69.372367,2060.164490,0.095390
BG0 0,"POLYGON ((29.34583 42.24111, 28.32389 41.99778...","(29.28183834594244, 42.87333189200004)",BG0 0,95.498412,0.148386,0.176485,87.582734,11052.570807,0.142613,45.281165,...,0.046705,31.907542,7.046495,0.053815,NaN,NaN,NaN,52.951120,3345.966155,0.124563
DE0 11,"POLYGON ((8.28604 53.42089, 8.24317 53.41079, ...","(6.336794924479278, 54.505438310008)",DE0 11,53.873868,0.669227,0.364074,44.498808,43594.793974,0.295848,62.960328,...,0.025180,17.588450,0.014318,0.018676,NaN,NaN,NaN,70.716513,0.226622,0.078537
DE0 15,"MULTIPOLYGON (((8.81986 54.19861, 8.81277 54.1...","(8.17932998604675, 54.53228697800006)",DE0 15,62.126430,0.790338,0.319996,52.030347,89617.262854,0.320665,62.003765,...,0.024803,29.140615,0.020005,0.020499,NaN,NaN,NaN,71.601688,0.880078,0.084488


In [41]:
df_exp_ons.head()

,geometry,coords,name,offwind-ac_mv,offwind-ac_gen,offwind-ac_cf,onwind_mv,onwind_gen,onwind_cf,solar_mv,...,BEV charger_con_el,BEV charger_cost_mv_el,battery charger_con_el,battery charger_cost_mv_el,urban central air heat pump_con_el,urban central air heat pump_cost_mv_el,urban central resistive heater_con_el,urban central resistive heater_cost_mv_el,PHS_con_el,PHS_cost_mv_el
name,,,,,,,,,,,,,,,,,,,,,
AL0 0,"POLYGON ((20.32207 39.91318, 20.39703 39.81809...","(20.00648437943388, 41.14563914300007)",AL0 0,NaN,0.016331,0.087899,NaN,0.016423,0.043458,43.422954,...,-2731.320599,61.820816,-5.597257,NaN,-183.066507,101.617344,-60.195311,83.481522,NaN,NaN
AT0 0,"POLYGON ((16.45660 47.41184, 16.43396 47.39685...","(15.700718667666258, 47.896990619088925)",AT0 0,NaN,NaN,NaN,79.253841,0.567355,0.144707,55.129859,...,-11683.551291,NaN,-0.991366,65.752773,-1957.988618,75.914663,-490.190809,63.791026,NaN,NaN
AT0 1,"POLYGON ((10.08288 47.35907, 10.20928 47.37248...","(10.379200993764837, 47.18231496200006)",AT0 1,NaN,NaN,NaN,NaN,0.000326,0.004923,57.245535,...,-1467.444465,81.263205,-0.793706,63.957909,-0.307180,74.711860,-56.277590,59.187063,-1905.537293,NaN
AT0 2,"POLYGON ((13.47802 46.56357, 13.23111 46.55217...","(13.805469952867842, 47.57283376100011)",AT0 2,NaN,NaN,NaN,NaN,0.012936,0.044168,54.745027,...,-7213.218328,81.186546,-1.248353,63.330463,-263.703344,76.701790,-244.231724,61.638273,-4224.086368,NaN
BA0 0,"POLYGON ((17.82716 42.85312, 17.81176 42.90986...","(17.97055162265331, 43.88773569750005)",BA0 0,NaN,NaN,NaN,NaN,0.010620,0.031748,51.151356,...,-2549.718336,57.625222,-1.457810,62.045594,-164.776325,78.079794,-67.563622,70.452362,NaN,NaN


In [42]:
df_exp_off.head()

,geometry,coords,name,offwind-ac_mv,offwind-ac_gen,offwind-ac_cf,onwind_mv,onwind_gen,onwind_cf,solar_mv,...,services urban decentral solar thermal_cf,urban central solar thermal_mv,urban central solar thermal_gen,urban central solar thermal_cf,oil_mv,oil_gen,oil_cf,solar rooftop_mv,solar rooftop_gen,solar rooftop_cf
name,,,,,,,,,,,,,,,,,,,,,
AL0 0,"POLYGON ((19.98542 39.69475, 20.01003 39.69120...","(19.04227857401803, 40.790848115835146)",AL0 0,106.977597,0.016331,0.087899,106.502516,0.016423,0.043458,43.422954,...,0.052464,27.814360,128.538879,0.062264,NaN,NaN,NaN,55.050254,3018.547275,0.130763
BE0 0,"POLYGON ((3.25367 51.34858, 3.10403 51.31485, ...","(2.728512798889029, 51.5008335)",BE0 0,56.352353,6343.446483,0.406369,52.212890,9585.550789,0.327713,58.383401,...,0.025774,29.938956,0.020416,0.027160,NaN,NaN,NaN,66.894575,1.726123,0.092860
BG0 0,"POLYGON ((29.34583 42.24111, 28.32389 41.99778...","(29.28183834594244, 42.87333189200004)",BG0 0,94.568790,0.100601,0.178239,85.769797,8.556417,0.144543,44.802486,...,0.048685,28.496283,1.715255,0.059337,NaN,NaN,NaN,51.305046,3363.626125,0.125219
DE0 11,"POLYGON ((8.28604 53.42089, 8.24317 53.41079, ...","(6.336794924479278, 54.505438310008)",DE0 11,51.455031,8441.804726,0.434748,47.257302,45018.091917,0.305507,60.991588,...,0.024480,33.265504,0.021125,0.026457,NaN,NaN,NaN,66.137206,0.083125,0.076325
DE0 15,"MULTIPOLYGON (((8.81986 54.19861, 8.81277 54.1...","(8.17932998604675, 54.53228697800006)",DE0 15,60.568652,0.306136,0.308317,50.245283,89980.590188,0.321965,61.958604,...,0.024536,31.269660,0.019227,0.025714,NaN,NaN,NaN,66.952632,0.143762,0.080772


In [43]:
# save to pickle
df_stst_ons.to_pickle("../data/processed/df_stst_ons.pkl")
df_stst_off.to_pickle("../data/processed/df_stst_off.pkl")
df_exp_ons.to_pickle("../data/processed/df_exp_ons.pkl")
df_exp_off.to_pickle("../data/processed/df_exp_off.pkl")

In [44]:
# verify reload
df_stst_ons = pd.read_pickle("../data/processed/df_stst_ons.pkl")
df_stst_off = pd.read_pickle("../data/processed/df_stst_off.pkl")
df_exp_ons = pd.read_pickle("../data/processed/df_exp_ons.pkl")
df_exp_off = pd.read_pickle("../data/processed/df_exp_off.pkl")

In [45]:
df_stst_ons.head()

,geometry,coords,name,offwind-ac_mv,offwind-ac_gen,offwind-ac_cf,onwind_mv,onwind_gen,onwind_cf,solar_mv,...,BEV charger_con_el,BEV charger_cost_mv_el,battery charger_con_el,battery charger_cost_mv_el,urban central air heat pump_con_el,urban central air heat pump_cost_mv_el,urban central resistive heater_con_el,urban central resistive heater_cost_mv_el,PHS_con_el,PHS_cost_mv_el
name,,,,,,,,,,,,,,,,,,,,,
AL0 0,"POLYGON ((20.32207 39.91318, 20.39703 39.81809...","(20.00648437943388, 41.14563914300007)",AL0 0,NaN,0.029238,0.090705,NaN,0.015125,0.038018,44.628736,...,-2904.296308,62.673081,-10501.207286,NaN,-161.142629,142.000494,-6.587224,51.370506,NaN,NaN
AT0 0,"POLYGON ((16.45660 47.41184, 16.43396 47.39685...","(15.700718667666258, 47.896990619088925)",AT0 0,NaN,NaN,NaN,83.457109,30395.904252,0.149667,54.679083,...,-11920.650630,NaN,-9.661850,66.778969,-1892.970068,96.915190,-1366.630022,NaN,NaN,NaN
AT0 1,"POLYGON ((10.08288 47.35907, 10.20928 47.37248...","(10.379200993764837, 47.18231496200006)",AT0 1,NaN,NaN,NaN,NaN,0.000348,0.004685,65.745903,...,-1495.945182,105.445665,-1880.142837,75.537145,-3.673844,90.315217,-108.354978,54.638572,-2221.062727,NaN
AT0 2,"POLYGON ((13.47802 46.56357, 13.23111 46.55217...","(13.805469952867842, 47.57283376100011)",AT0 2,NaN,NaN,NaN,NaN,0.014320,0.039178,58.223787,...,-7323.647076,96.649590,-5879.983158,NaN,-668.225567,92.091020,-777.763461,33.551977,-4907.238190,NaN
BA0 0,"POLYGON ((17.82716 42.85312, 17.81176 42.90986...","(17.97055162265331, 43.88773569750005)",BA0 0,NaN,NaN,NaN,NaN,0.014059,0.031802,51.160537,...,-2996.008222,58.325767,-4823.635061,NaN,-177.316568,96.225890,-96.014816,38.225192,NaN,NaN


In [46]:
df_stst_ons.head().dtypes

geometry                                     geometry
coords                                         object
name                                           object
offwind-ac_mv                                 float64
offwind-ac_gen                                float64
                                               ...   
urban central air heat pump_cost_mv_el        float64
urban central resistive heater_con_el         float64
urban central resistive heater_cost_mv_el     float64
PHS_con_el                                    float64
PHS_cost_mv_el                                float64
Length: 245, dtype: object

In [47]:
df_stst_off.head()

,geometry,coords,name,offwind-ac_mv,offwind-ac_gen,offwind-ac_cf,onwind_mv,onwind_gen,onwind_cf,solar_mv,...,services urban decentral solar thermal_cf,urban central solar thermal_mv,urban central solar thermal_gen,urban central solar thermal_cf,oil_mv,oil_gen,oil_cf,solar rooftop_mv,solar rooftop_gen,solar rooftop_cf
name,,,,,,,,,,,,,,,,,,,,,
AL0 0,"POLYGON ((19.98542 39.69475, 20.01003 39.69120...","(19.04227857401803, 40.790848115835146)",AL0 0,126.148417,0.029238,0.090705,131.075254,0.015125,0.038018,44.628736,...,0.049208,45.096866,73.985333,0.038386,NaN,NaN,NaN,56.499070,2987.307119,0.129410
BE0 0,"POLYGON ((3.25367 51.34858, 3.10403 51.31485, ...","(2.728512798889029, 51.5008335)",BE0 0,78.122119,6139.041623,0.393274,73.103671,9140.702915,0.312504,58.613241,...,0.025145,34.889495,0.044487,0.027616,NaN,NaN,NaN,69.372367,2060.164490,0.095390
BG0 0,"POLYGON ((29.34583 42.24111, 28.32389 41.99778...","(29.28183834594244, 42.87333189200004)",BG0 0,95.498412,0.148386,0.176485,87.582734,11052.570807,0.142613,45.281165,...,0.046705,31.907542,7.046495,0.053815,NaN,NaN,NaN,52.951120,3345.966155,0.124563
DE0 11,"POLYGON ((8.28604 53.42089, 8.24317 53.41079, ...","(6.336794924479278, 54.505438310008)",DE0 11,53.873868,0.669227,0.364074,44.498808,43594.793974,0.295848,62.960328,...,0.025180,17.588450,0.014318,0.018676,NaN,NaN,NaN,70.716513,0.226622,0.078537
DE0 15,"MULTIPOLYGON (((8.81986 54.19861, 8.81277 54.1...","(8.17932998604675, 54.53228697800006)",DE0 15,62.126430,0.790338,0.319996,52.030347,89617.262854,0.320665,62.003765,...,0.024803,29.140615,0.020005,0.020499,NaN,NaN,NaN,71.601688,0.880078,0.084488


In [48]:
df_exp_ons.head()

,geometry,coords,name,offwind-ac_mv,offwind-ac_gen,offwind-ac_cf,onwind_mv,onwind_gen,onwind_cf,solar_mv,...,BEV charger_con_el,BEV charger_cost_mv_el,battery charger_con_el,battery charger_cost_mv_el,urban central air heat pump_con_el,urban central air heat pump_cost_mv_el,urban central resistive heater_con_el,urban central resistive heater_cost_mv_el,PHS_con_el,PHS_cost_mv_el
name,,,,,,,,,,,,,,,,,,,,,
AL0 0,"POLYGON ((20.32207 39.91318, 20.39703 39.81809...","(20.00648437943388, 41.14563914300007)",AL0 0,NaN,0.016331,0.087899,NaN,0.016423,0.043458,43.422954,...,-2731.320599,61.820816,-5.597257,NaN,-183.066507,101.617344,-60.195311,83.481522,NaN,NaN
AT0 0,"POLYGON ((16.45660 47.41184, 16.43396 47.39685...","(15.700718667666258, 47.896990619088925)",AT0 0,NaN,NaN,NaN,79.253841,0.567355,0.144707,55.129859,...,-11683.551291,NaN,-0.991366,65.752773,-1957.988618,75.914663,-490.190809,63.791026,NaN,NaN
AT0 1,"POLYGON ((10.08288 47.35907, 10.20928 47.37248...","(10.379200993764837, 47.18231496200006)",AT0 1,NaN,NaN,NaN,NaN,0.000326,0.004923,57.245535,...,-1467.444465,81.263205,-0.793706,63.957909,-0.307180,74.711860,-56.277590,59.187063,-1905.537293,NaN
AT0 2,"POLYGON ((13.47802 46.56357, 13.23111 46.55217...","(13.805469952867842, 47.57283376100011)",AT0 2,NaN,NaN,NaN,NaN,0.012936,0.044168,54.745027,...,-7213.218328,81.186546,-1.248353,63.330463,-263.703344,76.701790,-244.231724,61.638273,-4224.086368,NaN
BA0 0,"POLYGON ((17.82716 42.85312, 17.81176 42.90986...","(17.97055162265331, 43.88773569750005)",BA0 0,NaN,NaN,NaN,NaN,0.010620,0.031748,51.151356,...,-2549.718336,57.625222,-1.457810,62.045594,-164.776325,78.079794,-67.563622,70.452362,NaN,NaN


In [49]:
df_exp_off.head()

,geometry,coords,name,offwind-ac_mv,offwind-ac_gen,offwind-ac_cf,onwind_mv,onwind_gen,onwind_cf,solar_mv,...,services urban decentral solar thermal_cf,urban central solar thermal_mv,urban central solar thermal_gen,urban central solar thermal_cf,oil_mv,oil_gen,oil_cf,solar rooftop_mv,solar rooftop_gen,solar rooftop_cf
name,,,,,,,,,,,,,,,,,,,,,
AL0 0,"POLYGON ((19.98542 39.69475, 20.01003 39.69120...","(19.04227857401803, 40.790848115835146)",AL0 0,106.977597,0.016331,0.087899,106.502516,0.016423,0.043458,43.422954,...,0.052464,27.814360,128.538879,0.062264,NaN,NaN,NaN,55.050254,3018.547275,0.130763
BE0 0,"POLYGON ((3.25367 51.34858, 3.10403 51.31485, ...","(2.728512798889029, 51.5008335)",BE0 0,56.352353,6343.446483,0.406369,52.212890,9585.550789,0.327713,58.383401,...,0.025774,29.938956,0.020416,0.027160,NaN,NaN,NaN,66.894575,1.726123,0.092860
BG0 0,"POLYGON ((29.34583 42.24111, 28.32389 41.99778...","(29.28183834594244, 42.87333189200004)",BG0 0,94.568790,0.100601,0.178239,85.769797,8.556417,0.144543,44.802486,...,0.048685,28.496283,1.715255,0.059337,NaN,NaN,NaN,51.305046,3363.626125,0.125219
DE0 11,"POLYGON ((8.28604 53.42089, 8.24317 53.41079, ...","(6.336794924479278, 54.505438310008)",DE0 11,51.455031,8441.804726,0.434748,47.257302,45018.091917,0.305507,60.991588,...,0.024480,33.265504,0.021125,0.026457,NaN,NaN,NaN,66.137206,0.083125,0.076325
DE0 15,"MULTIPOLYGON (((8.81986 54.19861, 8.81277 54.1...","(8.17932998604675, 54.53228697800006)",DE0 15,60.568652,0.306136,0.308317,50.245283,89980.590188,0.321965,61.958604,...,0.024536,31.269660,0.019227,0.025714,NaN,NaN,NaN,66.952632,0.143762,0.080772


## Temporal dfs

In [50]:
# calc electricity generation or consumption for every time step

df_stst_ts = pd.DataFrame(index=stst.generators_t.p.index)
df_exp_ts = pd.DataFrame(index=exp.generators_t.p.index)

for n, df in zip([stst, exp],[df_stst_ts, df_exp_ts]):
    for c in (c_el_gen_s + c_el_con_s):

        if c in n.generators.carrier.unique():
            df[f"{c}_gen"] = n.generators_t.p.loc[:, n.generators.carrier == c].sum(axis=1)

        elif c in n.links.carrier.unique():
            df[f"{c}_gen"] = -n.links_t.p1.loc[:, n.links.carrier == c].sum(axis=1)
            df[f"{c}_con_el"] = -n.links_t.p1.loc[:, n.links.carrier == c].sum(axis=1) * -1

        elif c in n.storage_units.carrier.unique():
            df[f"{c}_gen"] = n.storage_units_t.p_dispatch.loc[:, n.storage_units.carrier == c].sum(axis=1)
            df[f"{c}_con_el"] = n.storage_units_t.p_store.loc[:, n.storage_units.carrier == c].sum(axis=1) * -1

        else:
            print(f"{c} is not a known carrier")

In [51]:
df_stst_ts.head()

,onwind_gen,solar_gen,offwind-dc_gen,hydro_gen,hydro_con_el,solar rooftop_gen,battery discharger_gen,battery discharger_con_el,offwind-ac_gen,ror_gen,...,H2 Electrolysis_gen,H2 Electrolysis_con_el,BEV charger_gen,BEV charger_con_el,battery charger_gen,battery charger_con_el,urban central air heat pump_gen,urban central air heat pump_con_el,urban central resistive heater_gen,urban central resistive heater_con_el
snapshot,,,,,,,,,,,,,,,,,,,,,
2013-01-01 00:00:00,1.115046e+06,0.000000,96780.482471,32616.407352,-0.0,0.000000,41299.585753,-41299.585753,36717.442882,12320.808629,...,472983.016887,-472983.016887,141501.650312,-141501.650312,9003.183588,-9003.183588,121154.555872,-121154.555872,16023.457529,-16023.457529
2013-01-01 03:00:00,1.055760e+06,0.000000,87476.404567,34583.787290,-0.0,0.000000,50472.110616,-50472.110616,30103.183830,12352.313154,...,431130.292709,-431130.292709,101838.924102,-101838.924102,10103.946175,-10103.946175,122281.212604,-122281.212604,45365.287424,-45365.287424
2013-01-01 06:00:00,9.886784e+05,158083.842075,99085.284393,35854.176137,-0.0,17369.631860,13962.508203,-13962.508203,30304.778869,12489.446573,...,418300.065034,-418300.065034,137797.551009,-137797.551009,33412.155433,-33412.155433,122899.974911,-122899.974911,55752.405009,-55752.405009
2013-01-01 09:00:00,8.329774e+05,831849.855205,106101.418311,24616.469476,-0.0,77017.545359,271.323747,-271.323747,27248.506958,12641.889286,...,616780.479643,-616780.479643,248275.864754,-248275.864754,153979.285016,-153979.285016,126537.038034,-126537.038034,49023.531577,-49023.531577
2013-01-01 12:00:00,8.327530e+05,652761.327098,111841.816426,42924.819243,-0.0,57996.306341,1935.111795,-1935.111795,27101.491865,12971.781726,...,548648.824131,-548648.824131,221648.035099,-221648.035099,141436.903434,-141436.903434,129377.063056,-129377.063056,38437.555919,-38437.555919


In [52]:
df_exp_ts.head()

,onwind_gen,solar_gen,offwind-dc_gen,hydro_gen,hydro_con_el,solar rooftop_gen,battery discharger_gen,battery discharger_con_el,offwind-ac_gen,ror_gen,...,H2 Electrolysis_gen,H2 Electrolysis_con_el,BEV charger_gen,BEV charger_con_el,battery charger_gen,battery charger_con_el,urban central air heat pump_gen,urban central air heat pump_con_el,urban central resistive heater_gen,urban central resistive heater_con_el
snapshot,,,,,,,,,,,,,,,,,,,,,
2013-01-01 00:00:00,1.004871e+06,0.000000,132715.807997,16254.378260,-0.0,0.000000,16378.782579,-16378.782579,26814.308286,12359.133966,...,383673.410285,-383673.410285,130743.122562,-130743.122562,22.472782,-22.472782,119774.414877,-119774.414877,25279.984642,-25279.984642
2013-01-01 03:00:00,9.464829e+05,0.000000,119299.611895,16316.852435,-0.0,0.000000,38152.768816,-38152.768816,20431.056746,12390.526455,...,344086.285332,-344086.285332,71910.298594,-71910.298594,19.624660,-19.624660,118784.842481,-118784.842481,65392.837163,-65392.837163
2013-01-01 06:00:00,8.902786e+05,123481.585163,141140.696947,406.617021,-0.0,15266.921462,724.352564,-724.352564,20830.104695,12524.626379,...,306165.039068,-306165.039068,117902.017322,-117902.017322,6551.860796,-6551.860796,118205.728930,-118205.728930,79231.035858,-79231.035858
2013-01-01 09:00:00,7.909976e+05,605243.173189,157478.679277,404.648028,-0.0,65890.035274,12.581618,-12.581618,19272.243401,12710.149548,...,491742.370063,-491742.370063,224795.950456,-224795.950456,54945.071967,-54945.071967,122076.629832,-122076.629832,65018.567538,-65018.567538
2013-01-01 12:00:00,7.799514e+05,490742.596278,173255.846806,423.282346,-0.0,50699.632594,16.483854,-16.483854,19832.735058,12973.790312,...,441456.912589,-441456.912589,203407.228199,-203407.228199,54172.258425,-54172.258425,125193.019625,-125193.019625,56782.152060,-56782.152060


In [53]:
# save to pickle
df_stst_ts.to_pickle("../data/processed/df_stst_ts.pkl")
df_exp_ts.to_pickle("../data/processed/df_exp_ts.pkl")

# verify reload
df_stst_ts = pd.read_pickle("../data/processed/df_stst_ts.pkl")
df_exp_ts = pd.read_pickle("../data/processed/df_exp_ts.pkl")

In [54]:
df_stst_ts.head()

,onwind_gen,solar_gen,offwind-dc_gen,hydro_gen,hydro_con_el,solar rooftop_gen,battery discharger_gen,battery discharger_con_el,offwind-ac_gen,ror_gen,...,H2 Electrolysis_gen,H2 Electrolysis_con_el,BEV charger_gen,BEV charger_con_el,battery charger_gen,battery charger_con_el,urban central air heat pump_gen,urban central air heat pump_con_el,urban central resistive heater_gen,urban central resistive heater_con_el
snapshot,,,,,,,,,,,,,,,,,,,,,
2013-01-01 00:00:00,1.115046e+06,0.000000,96780.482471,32616.407352,-0.0,0.000000,41299.585753,-41299.585753,36717.442882,12320.808629,...,472983.016887,-472983.016887,141501.650312,-141501.650312,9003.183588,-9003.183588,121154.555872,-121154.555872,16023.457529,-16023.457529
2013-01-01 03:00:00,1.055760e+06,0.000000,87476.404567,34583.787290,-0.0,0.000000,50472.110616,-50472.110616,30103.183830,12352.313154,...,431130.292709,-431130.292709,101838.924102,-101838.924102,10103.946175,-10103.946175,122281.212604,-122281.212604,45365.287424,-45365.287424
2013-01-01 06:00:00,9.886784e+05,158083.842075,99085.284393,35854.176137,-0.0,17369.631860,13962.508203,-13962.508203,30304.778869,12489.446573,...,418300.065034,-418300.065034,137797.551009,-137797.551009,33412.155433,-33412.155433,122899.974911,-122899.974911,55752.405009,-55752.405009
2013-01-01 09:00:00,8.329774e+05,831849.855205,106101.418311,24616.469476,-0.0,77017.545359,271.323747,-271.323747,27248.506958,12641.889286,...,616780.479643,-616780.479643,248275.864754,-248275.864754,153979.285016,-153979.285016,126537.038034,-126537.038034,49023.531577,-49023.531577
2013-01-01 12:00:00,8.327530e+05,652761.327098,111841.816426,42924.819243,-0.0,57996.306341,1935.111795,-1935.111795,27101.491865,12971.781726,...,548648.824131,-548648.824131,221648.035099,-221648.035099,141436.903434,-141436.903434,129377.063056,-129377.063056,38437.555919,-38437.555919


In [55]:
df_exp_ts.head()

,onwind_gen,solar_gen,offwind-dc_gen,hydro_gen,hydro_con_el,solar rooftop_gen,battery discharger_gen,battery discharger_con_el,offwind-ac_gen,ror_gen,...,H2 Electrolysis_gen,H2 Electrolysis_con_el,BEV charger_gen,BEV charger_con_el,battery charger_gen,battery charger_con_el,urban central air heat pump_gen,urban central air heat pump_con_el,urban central resistive heater_gen,urban central resistive heater_con_el
snapshot,,,,,,,,,,,,,,,,,,,,,
2013-01-01 00:00:00,1.004871e+06,0.000000,132715.807997,16254.378260,-0.0,0.000000,16378.782579,-16378.782579,26814.308286,12359.133966,...,383673.410285,-383673.410285,130743.122562,-130743.122562,22.472782,-22.472782,119774.414877,-119774.414877,25279.984642,-25279.984642
2013-01-01 03:00:00,9.464829e+05,0.000000,119299.611895,16316.852435,-0.0,0.000000,38152.768816,-38152.768816,20431.056746,12390.526455,...,344086.285332,-344086.285332,71910.298594,-71910.298594,19.624660,-19.624660,118784.842481,-118784.842481,65392.837163,-65392.837163
2013-01-01 06:00:00,8.902786e+05,123481.585163,141140.696947,406.617021,-0.0,15266.921462,724.352564,-724.352564,20830.104695,12524.626379,...,306165.039068,-306165.039068,117902.017322,-117902.017322,6551.860796,-6551.860796,118205.728930,-118205.728930,79231.035858,-79231.035858
2013-01-01 09:00:00,7.909976e+05,605243.173189,157478.679277,404.648028,-0.0,65890.035274,12.581618,-12.581618,19272.243401,12710.149548,...,491742.370063,-491742.370063,224795.950456,-224795.950456,54945.071967,-54945.071967,122076.629832,-122076.629832,65018.567538,-65018.567538
2013-01-01 12:00:00,7.799514e+05,490742.596278,173255.846806,423.282346,-0.0,50699.632594,16.483854,-16.483854,19832.735058,12973.790312,...,441456.912589,-441456.912589,203407.228199,-203407.228199,54172.258425,-54172.258425,125193.019625,-125193.019625,56782.152060,-56782.152060
